## Data: Head position

In [1]:
# Create your first MLP in Keras
import os, sys
import numpy as np
import time
from scipy import interpolate
from utils.convert_coord import equirect_to_cart, cart_to_equirect
from utils.data_generator import data_generator_head_clean
import matplotlib.pyplot as plt
%matplotlib inline
import random as rn
import statsmodels.api as sm

dir_path = os.getcwd()

In [2]:
experiment_name = 'seq2seq_linear_regression'

time_step = 10 # 10 ms
lookback = 250 # 250 ms
n_lookback = int(round(lookback / time_step))

step_delay = 100
n_delay = int(round(step_delay / time_step))
delays_list = list(range(100, 1000+1, step_delay))
n_max_delay = int(round(np.max(delays_list) / time_step))

len_delay = len(delays_list)

In [3]:
saliency_dir = os.path.join(dir_path, "fov_images", "saliency")
#videos = os.listdir(saliency_dir) 
#print(videos)

path_to_sensor_data_train = os.path.join(dir_path, "preprocessed_train")
videos = os.listdir(path_to_sensor_data_train)
videos = videos
print(videos)
video_width = 3840

['013_13_times_square_static.npy', '025_07_factory_robots_static.npy', '002_01_art_gallery_static.npy', '002_02_theatre_scene_static.npy', '002_03_lions_static_cuts.npy', '002_04_copenhagen_harbour_static_cuts.npy', '002_07_factory_robots_static.npy', '002_09_elephants_static.npy', '002_12_gym_workout_static.npy', '002_13_times_square_static.npy', '002_14_new_orleans_drive_motion.npy', '002_16_interview_static.npy', '002_17_weather_forecast_static.npy', '002_18_bomb_trapped_static.npy', '002_20_car_fix_static.npy', '002_23_dog_food_factory_motion.npy', '003_01_art_gallery_static.npy', '003_02_theatre_scene_static.npy', '003_03_lions_static_cuts.npy', '003_04_copenhagen_harbour_static_cuts.npy', '003_05_florida_yacht_motion.npy', '003_07_factory_robots_static.npy', '003_08_madagascar_cuts.npy', '003_09_elephants_static.npy', '003_11_lohdi_garden_india_cuts.npy', '003_12_gym_workout_static.npy', '003_13_times_square_static.npy', '003_14_new_orleans_drive_motion.npy', '003_18_bomb_trapped

In [4]:
use_generator = True
if use_generator:
    nb_epochs = 150
    start = time.time()
    
    perc_train = 0.8
    ind_max_train = int(np.round(len(videos) * perc_train))
    file_names_train = videos[:ind_max_train]
    file_names_val = videos[ind_max_train:]
    #data = np.load(os.path.join(path_to_sensor_data_train, file_name))

    train_gen = data_generator_head_clean(path_to_sensor_data_train,
                                    file_names_train,
                                    delays_list,
                                    n_max_delay,
                                    n_lookback,
                                    n_delay)
    val_gen = data_generator_head_clean(path_to_sensor_data_train,
                                  file_names_val,
                                  delays_list,
                                  n_max_delay,
                                  n_lookback,
                                  n_delay,
                                  inference=True
                                 )

In [5]:
#path_to_sensor_data_train = os.path.join(dir_path, "preprocessed_train")

start = time.time()

X_x_diff_train_final = None
X_y_diff_train_final = None
X_z_diff_train_final = None

Y_x_diff_train_final = None
Y_y_diff_train_final = None
Y_z_diff_train_final = None

count = 0

for [X_x_diff_train, X_y_diff_train, X_z_diff_train], Y_diff_train in train_gen:
    print(count)
    if X_x_diff_train_final is None:
        X_x_diff_train_final = X_x_diff_train
        X_y_diff_train_final = X_y_diff_train
        X_z_diff_train_final = X_z_diff_train
        
        Y_x_diff_train_final = Y_diff_train[:, 0:10]
        Y_y_diff_train_final = Y_diff_train[:, 10:20]
        Y_z_diff_train_final = Y_diff_train[:, 20:30]
        
    else:
        X_x_diff_train_final = np.concatenate((X_x_diff_train_final, X_x_diff_train), axis=0)
        X_y_diff_train_final = np.concatenate((X_y_diff_train_final, X_y_diff_train), axis=0)
        X_z_diff_train_final = np.concatenate((X_z_diff_train_final, X_z_diff_train), axis=0)
        
        Y_x_diff_train_final = np.concatenate((Y_x_diff_train_final, Y_diff_train[:, 0:10]), axis=0)
        Y_y_diff_train_final = np.concatenate((Y_y_diff_train_final, Y_diff_train[:, 10:20]), axis=0)
        Y_z_diff_train_final = np.concatenate((Y_z_diff_train_final, Y_diff_train[:, 20:30]), axis=0)
    
    count += 1
    if count == len(file_names_train):
        break

print("Evaluation Time : ", time.time() - start, "s = ", (time.time() - start)/60., "min = ",
      (time.time() - start)/3600., "h")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
print(X_x_diff_train_final.shape)
print(Y_x_diff_train_final.shape)
# Note the diff_trainerence in argument order
model_x = sm.OLS(Y_x_diff_train_final, X_x_diff_train_final[:,:,0]).fit()
model_y = sm.OLS(Y_y_diff_train_final, X_y_diff_train_final[:,:,0]).fit()
model_z = sm.OLS(Y_z_diff_train_final, X_z_diff_train_final[:,:,0]).fit()

(3185306, 25, 1)
(3185306, 10)


In [13]:
#path_to_sensor_data_val = os.path.join(dir_path, "preprocessed_val")

start = time.time()
print("Evaluating on validation data...")
count = 0

X_x_diff_val_final = None
X_y_diff_val_final = None
X_z_diff_val_final = None

Y_abs_val_final = None

for [X_x_diff_val, X_y_diff_val, X_z_diff_val], Y_abs_val in val_gen:
    
    if X_x_diff_val_final is None:
        X_x_diff_val_final = X_x_diff_val
        X_y_diff_val_final = X_y_diff_val
        X_z_diff_val_final = X_z_diff_val
        
        Y_abs_val_final = Y_abs_val
        
    else:
        X_x_diff_val_final = np.concatenate((X_x_diff_val_final, X_x_diff_val), axis=0)
        X_y_diff_val_final = np.concatenate((X_y_diff_val_final, X_y_diff_val), axis=0)
        X_z_diff_val_final = np.concatenate((X_z_diff_val_final, X_z_diff_val), axis=0)
        
        Y_abs_val_final = np.concatenate((Y_abs_val_final, Y_abs_val), axis=0)
    
    count += 1
    if count == len(file_names_val):
        break

print("Evaluation Time : ", time.time() - start, "s = ", (time.time() - start)/60., "min = ",
      (time.time() - start)/3600., "h")

Evaluating on validation data...
Evaluation Time :  24.902502298355103 s =  0.415041716893514 min =  0.006917362014452616 h


In [34]:
# HARD CODED MAXIMUM DIFFERENCES FOR TRAINING SET!
max_diff = np.array([1.67219621, 0.06364631, 1.81424042])

Y_x_diff_val_pred = model_x.predict(X_x_diff_val_final[:,:,0]) # make the predictions by the model
Y_y_diff_val_pred = model_y.predict(X_y_diff_val_final[:,:,0]) # make the predictions by the model
Y_z_diff_val_pred = model_z.predict(X_z_diff_val_final[:,:,0]) # make the predictions by the model

Y_abs_val_pred = np.zeros(shape=(Y_x_diff_val_pred.shape[0], Y_x_diff_val_pred.shape[1], 3))


for i in range(Y_x_diff_val_pred.shape[1]):
    # print(Y_abs_val_pred[:, i, 0].shape, Y_abs_val_final[:, 0, 0].shape, np.sum(Y_x_diff_val_pred[:, :i+1], axis=1).shape)
    Y_abs_val_pred[:, i, 0] = Y_abs_val_final[:, 0, 0] + np.sum(Y_x_diff_val_pred[:, :i+1], axis=1) * max_diff[0]
    Y_abs_val_pred[:, i, 1] = Y_abs_val_final[:, 0, 1] + np.sum(Y_y_diff_val_pred[:, :i+1], axis=1) * max_diff[1]
    Y_abs_val_pred[:, i, 2] = Y_abs_val_final[:, 0, 2] + np.sum(Y_z_diff_val_pred[:, :i+1], axis=1) * max_diff[2]
    
Y_abs_val_final = Y_abs_val_final[:, 1:]

In [46]:
def get_mae(Y1, Y2):
    return np.mean(np.absolute(Y1 - Y2))

def get_rmse(Y1, Y2):
    return np.sqrt(np.mean(np.square(Y1 - Y2)))

def get_smape(Y1, Y2):
    return 100 * np.mean(np.absolute(Y1 - Y2) / (np.absolute(Y2) + np.absolute(Y1)))

def get_mfe(Y_pred, Y_target):
    return np.mean(Y_pred - Y_target)

#print(Y_abs_val_pred.shape, Y_abs_val_final.shape)

print("MAE")
print("Average mae:", get_mae(Y_abs_val_pred, Y_abs_val_final))
for ind_delay in range(0,len_delay):
    Y_delay_pred = Y_abs_val_pred[: ,ind_delay]
    Y_delay_val = Y_abs_val_final[: ,ind_delay]
    
    print("Delay", 100+ind_delay*100, "ms:", get_mae(Y_delay_pred, Y_delay_val))
print()
print("RMSE")
print("Average rmse:", get_rmse(Y_abs_val_pred, Y_abs_val_final))
for ind_delay in range(0,len_delay):
    Y_delay_pred = Y_abs_val_pred[: ,ind_delay]
    Y_delay_val = Y_abs_val_final[: ,ind_delay]
    
    print("Delay", 100+ind_delay*100, "ms:", get_rmse(Y_delay_pred, Y_delay_val))
print()
print("SMAPE")
print("Average smape:", get_smape(Y_abs_val_pred, Y_abs_val_final))
for ind_delay in range(0,len_delay):
    Y_delay_pred = Y_abs_val_pred[: ,ind_delay]
    Y_delay_val = Y_abs_val_final[: ,ind_delay]
    
    print("Delay", 100+ind_delay*100, "ms:", get_smape(Y_delay_pred, Y_delay_val))
print()
print("MFE")
print("Average mfe:", get_mfe(Y_abs_val_pred, Y_abs_val_final))
for ind_delay in range(0,len_delay):
    Y_delay_pred = Y_abs_val_pred[: ,ind_delay]
    Y_delay_val = Y_abs_val_final[: ,ind_delay]
    
    print("Delay", 100+ind_delay*100, "ms:", get_mfe(Y_delay_pred, Y_delay_val))

MAE
Average mae: 0.06515810167328699
Delay 100 ms: 0.006348098273935669
Delay 200 ms: 0.016490279693447685
Delay 300 ms: 0.02868225167907382
Delay 400 ms: 0.042008726911874704
Delay 500 ms: 0.05617501885217864
Delay 600 ms: 0.07086577290669573
Delay 700 ms: 0.08576578203675597
Delay 800 ms: 0.1006039957680197
Delay 900 ms: 0.11520447325700207
Delay 1000 ms: 0.12943661735388554

RMSE
Average rmse: 0.14548810084397262
Delay 100 ms: 0.019272919603214592
Delay 200 ms: 0.040998158512533515
Delay 300 ms: 0.06511789194984163
Delay 400 ms: 0.09001108630650038
Delay 500 ms: 0.11533502032932494
Delay 600 ms: 0.1406417349102267
Delay 700 ms: 0.16551029182609048
Delay 800 ms: 0.18967371857810214
Delay 900 ms: 0.2130219851220486
Delay 1000 ms: 0.23546302905652988

SMAPE
Average smape: 14.447705133330471
Delay 100 ms: 2.5073095012660325
Delay 200 ms: 5.5105019722501085
Delay 300 ms: 8.450620633322123
Delay 400 ms: 11.197236205478553
Delay 500 ms: 13.80594018254793
Delay 600 ms: 16.270661288121904
De